In [2]:
path = '7x7\maps'
extension = '.npy'

import numpy as np
import os

path = '7x7\maps'
extension = '.npy'

# import the numpy array
def import_map(map_name):
    map_path = os.path.join(path, str(map_name) + extension)
    return np.load(map_path)

a = import_map(0)
np.set_printoptions(threshold=np.inf)

print(a.shape)

FileNotFoundError: [Errno 2] No such file or directory: '7x7\\maps/0.npy'

In [12]:
b = np.zeros((2, 7, 7), dtype=int)
b

array([[[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]])

In [5]:
thresholds_array = np.array([0.2, 0.38, 0.5, 0.7, 0.9, 1.0])
print(thresholds_array[0])

0.2


In [ ]:
    for i in range(10):
        # Create a new heightmap and save it to a file in the folder '7x7'. The images in a subfolder named '7x7/img' and the heightmaps in a subfolder named '7x7/maps'
        heightmap_generator = HeightmapGenerator(width=350, height=350, scale=5, thresholds=thresholds, octaves=3, persistence=0.2, lacunarity=1.0)
        heightmap = heightmap_generator.generate()
        heightmap_generator.save_heightmap(f"7x7/maps/{i}.npy")
        renderer = TerrainRenderer(heightmap, colors, tile_size=50)
        # renderer.real_time_update()
        renderer.save_image(f"7x7/img/{i}.png")